# Ollama Python Library

The Ollama Python library provides the easiest way to integrate Python 3.8+ projects with Ollama.

## Prerequisites

- Ollama should be installed and running
- Pull a model to use with the library: ollama pull <model> e.g. ollama pull llama3.2
  - See Ollama.com for more information on the models available.

In [1]:
%pip install python-dotenv

%pip install ollama
%pip install boto3
%pip install tqdm

Note: you may need to restart the kernel to use updated packages.
  Using cached ollama-0.4.7-py3-none-any.whl.metadata (4.7 kB)
Using cached ollama-0.4.7-py3-none-any.whl (13 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
   ---------------------------------------- 0.0/13.4 MB ? eta -:--:--
   --- ------------------------------------ 1.3/13.4 MB 8.4 MB/s eta 0:00:02
   ----------- ---------------------------- 3.9/13.4 MB 10.2 MB/s eta 0:00:01
   --------------------- ------------------ 7.3/13.4 MB 12.2 MB/s eta 0:00:01
   --------------------------------- ------ 11.3/13.4 MB 14.1 MB/s eta 0:00:01
   ---------------------------------------- 13.4/13.4 MB 14.2 MB/s eta 0:00:00
Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
Note: you may need to restart the kernel to use updated pa

## Récuperation des variables d'environement

Pour le notebook, obligation d'utiliser la bibliotheque python-dotenv pour récuperer les varable d'environnment dans le .env.local 

In [1]:
import os
from dotenv import load_dotenv

load_dotenv("../.env.local")

print(os.getenv('AWS_ACCESS_KEY_ID'))
print(os.getenv('AWS_SECRET_ACCESS_KEY'))
print(os.getenv('AWS_REGION'))

None
None
None


## Connexion avec AWS et intéraction avec S3

In [ ]:
import os
import boto3
import logging
from typing import BinaryIO, List
from botocore.exceptions import ClientError

class S3:
    def __init__(self):
        self.s3_client = boto3.client(
            's3',
            aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
            aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
            region_name=os.getenv('AWS_REGION', 'eu-west-3')
        )
        self.s3_resource = boto3.resource(
            's3',
            aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
            aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
            region_name=os.getenv('AWS_REGION', 'eu-west-3')
        )
        self.bucket_name = 'hetic-web3-groupe11-mmotors'
        self.base_url = f"https://{self.bucket_name}.s3.{os.getenv('AWS_REGION', 'eu-west-3')}.amazonaws.com"

    def list_tree(self) -> List[str]:
        try:
            bucket = self.s3_resource.Bucket(self.bucket_name)
            return [obj.key for obj in bucket.objects.all()]
        except ClientError as e:
            logging.error(f"Erreur lors de la récupération de l'arborescence : {e}")
            raise

    def list_objects(self, folder_name: str = "") -> List[str]:
        try:
            bucket = self.s3_resource.Bucket(self.bucket_name)
            return [obj.key for obj in bucket.objects.filter(Prefix=folder_name)]
        except ClientError as e:
            logging.error(f"Erreur lors de la récupération des fichiers du dossier {folder_name} : {e}")
            raise

    def upload_file(self, file: BinaryIO, file_name: str, folder_name: str = "") -> str:
        try:
            key = f"{folder_name}/{file_name}" if folder_name else file_name
            self.s3_client.upload_fileobj(file, self.bucket_name, key)
            return f"{self.base_url}/{key}"
        except ClientError as e:
            logging.error(f"Erreur lors du téléversement de {file_name} : {e}")
            raise

    def download_file(self, file_name: str, folder_name: str = "", local_path: str = None) -> str:
        try:
            key = f"{folder_name}/{file_name}" if folder_name else file_name
            local_path = local_path or file_name
            self.s3_client.download_file(self.bucket_name, key, local_path)
            return local_path
        except ClientError as e:
            logging.error(f"Erreur lors du téléchargement de {file_name} : {e}")
            raise

    def download_all_files(self, folder: str, local_dir: str = "."):
        """
        Télécharge tous les fichiers d'un dossier S3 vers un répertoire local.
        :param folder: Nom du dossier (ex: 'mon-dossier/')
        :param local_dir: Dossier local où stocker les fichiers
        """
        try:
            # Vérifier et créer le dossier avec les bonnes permissions
            os.makedirs(local_dir, exist_ok=True)
            os.chmod(local_dir, 0o777)  # Assurez-vous que le dossier est accessible

            objects = self.list_objects(folder)
            if not objects:
                logging.warning(f"Aucun fichier trouvé dans {folder}. Vérifiez le préfixe.")
                return

            for obj in objects:
                file_name = obj.split("/")[-1]
                file_path = os.path.join(local_dir, file_name)
                
                logging.info(f"Téléchargement de {obj} vers {file_path}")
                self.download_file(obj, "", file_path)

        except PermissionError:
            logging.error(f"Permission refusée pour écrire dans {local_dir}. Vérifiez les droits d'accès.")
        except ClientError as e:
            logging.error(f"Erreur lors du téléchargement des fichiers de {folder} : {e}")
            raise
        
    def delete(self, file_name: str = None, folder_name: str = None) -> bool:
        try:
            if file_name:
                key = f"{folder_name}/{file_name}" if folder_name else file_name
                self.s3_client.delete_object(Bucket=self.bucket_name, Key=key)
                return True
            elif folder_name:
                for obj in self.list_objects(folder_name):
                    self.s3_client.delete_object(Bucket=self.bucket_name, Key=obj)
                return True
            return False
        except ClientError as e:
            logging.error(f"Erreur lors de la suppression : {e}")
            return False

### Instanciation de S3

In [ ]:
s3 = S3()

### Lister tous les fichiers du bucket

In [22]:
files = s3.list_tree()
print("Fichiers dans le bucket :", files)

Fichiers dans le bucket : ['development/', 'env-files/', 'env-files/aws-var.env', 'llm/', 'production/', 'test.txt']


###  Lister les fichiers dans un dossier spécifique

In [55]:
files_in_folder = s3.list_objects("llm/")
print("Fichiers dans le dossier 'llm/' :", files_in_folder)

Fichiers dans le dossier 'llm/' : ['llm/', 'llm/test.pdf']


### Upload d’un fichier dans un dossier spécifique

In [24]:
with open("./test.pdf", "rb") as file:
    file_url = s3.upload_file(file, "test.pdf", "llm")
    print(f"Fichier uploadé à : {file_url}")

Fichier uploadé à : https://hetic-web3-groupe11-mmotors.s3.eu-west-3.amazonaws.com/llm/test.pdf


### Télécharger un fichier depuis S3

In [26]:
s3.download_file("test.pdf", "llm", "test2.pdf")
print("Fichier téléchargé avec succès.")

Fichier téléchargé avec succès.


### Télécharger tous les fichiers d’un dossier S3

In [38]:
s3.download_all_files("llm", "local_llm/")
print("Tous les fichiers du dossier 'llm/' ont été téléchargés.")

PermissionError: [WinError 5] Accès refusé: 'local_llm/'

### Supprimer un fichier

In [ ]:
s3.delete("test.pdf", "llm")
print("Fichier supprimé.")

## Traitement du formats des documents

1. Formats de fichiers à traiter


| Format de fichier | Outil Python recommandé | Description |
|------------------|------------------------|-------------|
| **PDF** | `PyMuPDF (fitz)` | Extraction de texte et d'images à partir de fichiers PDF. |
| **CSV** | `pandas` | Lecture, transformation et analyse de données tabulaires. |
| **TXT** | `open()`, `io` | Lecture et manipulation simple de texte brut. |
| **HTML** | `BeautifulSoup` | Extraction de texte structuré à partir de pages web. |
| **JSON** | `orjson` | Parsing et manipulation de données semi-structurées. |
| **DOCX** | `python-docx` | Extraction et modification de texte dans des fichiers Microsoft Word. |
| **Markdown (MD)** | `mistune` | Parsing et conversion de fichiers Markdown en texte brut ou HTML. |
| **XML** | `lxml` | Parsing et extraction de contenu depuis des fichiers XML. |


In [3]:
%pip install PyMuPDF
%pip install beautifulsoup4
%pip install python-docx
%pip install mistune
%pip install panda
%pip install orjson
%pip install lxml

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import re
import fitz

def extract_text_from_pdf(pdf_path):
    text = ""
    with fitz.open(pdf_path) as doc:
        for page in doc:
            text += page.get_text("text") + "\n"
    text = re.sub(r'-\n', '', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

pdf = extract_text_from_pdf('./ref/doc.pdf')
print(pdf)

Guide d'achat et de location de voiture Introduction L'achat ou la location d'une voiture représente une décision importante qui dépend de nombreux critères : budget, usage, coûts d'entretien, valeur de revente, et préférences personnelles. Ce guide vous aidera à choisir la meilleure option en fonction de vos besoins et de votre situation financière. Partie 1 : Acheter une voiture 1.1 Avantages de l'achat - Liberté totale : Vous êtes propriétaire du véhicule et pouvez l'utiliser sans restriction. - Coûts à long terme réduits : Une fois le prêt remboursé (si achat à crédit), le coût mensuel diminue. - Valeur de revente : Possibilité de revendre le véhicule pour financer un futur achat. 1.2 Inconvénients de l'achat - Investissement initial élevé : Coût d'achat et d'immatriculation parfois importants. - Dépréciation rapide : Une voiture perd en moyenne 20 % de sa valeur la première année. - Coûts d'entretien et de réparation : À la charge du propriétaire. 1.3 Neuf ou occasion ? - Voiture 

In [ ]:
import re

def extract_text_from_txt(txt_path):
    with open(txt_path, "r", encoding="utf-8") as f:
        text = f.read()
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

txt = extract_text_from_txt('./ref/doc.txt')
print(txt)

"Guide d'achat et de location de voiture Introduction L'achat ou la location d'une voiture représente une décision importante qui dépend de nombreux critères : budget, usage, coûts d'entretien, valeur de revente, et préférences personnelles. Ce guide vous aidera à choisir la meilleure option en fonction de vos besoins et de votre situation financière. Partie 1 : Acheter une voiture 1.1 Avantages de l'achat - Liberté totale : Vous êtes propriétaire du véhicule et pouvez l'utiliser sans restriction. - Coûts à long terme réduits : Une fois le prêt remboursé (si achat à crédit), le coût mensuel diminue. - Valeur de revente : Possibilité de revendre le véhicule pour financer un futur achat. 1.2 Inconvénients de l'achat - Investissement initial élevé : Coût d'achat et d'immatriculation parfois importants. - Dépréciation rapide : Une voiture perd en moyenne 20 % de sa valeur la première année. - Coûts d'entretien et de réparation : À la charge du propriétaire. 1.3 Neuf ou occasion ? - Voiture

In [ ]:
import re
from bs4 import BeautifulSoup

def extract_text_from_html(html_path):
    with open(html_path, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "html.parser")
    text = soup.get_text(separator=" ")
    text = re.sub(r'\s+', ' ', text)
    return text.strip()


html = extract_text_from_html('./ref/doc.html')
print(html)

"Guide d'achat et de location de voiture Guide d'achat et de location de voiture Introduction L'achat ou la location d'une voiture représente une décision importante qui dépend de nombreux critères : budget, usage, coûts d'entretien, valeur de revente, et préférences personnelles. Ce guide vous aidera à choisir la meilleure option en fonction de vos besoins et de votre situation financière. Partie 1 : Acheter une voiture 1.1 Avantages de l'achat Liberté totale : Vous êtes propriétaire du véhicule et pouvez l'utiliser sans restriction. Coûts à long terme réduits : Une fois le prêt remboursé (si achat à crédit), le coût mensuel diminue. Valeur de revente : Possibilité de revendre le véhicule pour financer un futur achat. 1.2 Inconvénients de l'achat Investissement initial élevé : Coût d'achat et d'immatriculation parfois importants. Dépréciation rapide : Une voiture perd en moyenne 20 % de sa valeur la première année. Coûts d'entretien et de réparation : À la charge du propriétaire. 1.3 

In [ ]:
import re
from docx import Document

def extract_text_from_docx(docx_path):
    doc = Document(docx_path)
    text = "\n".join([p.text for p in doc.paragraphs if p.text.strip()])
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

docs = extract_text_from_docx('./ref/doc.docx')
print(docs)

"Guide d'achat et de location de voiture Introduction L'achat ou la location d'une voiture représente une décision importante qui dépend de nombreux critères : budget, usage, coûts d'entretien, valeur de revente, et préférences personnelles. Ce guide vous aidera à choisir la meilleure option en fonction de vos besoins et de votre situation financière. Partie 1 : Acheter une voiture 1.1 Avantages de l'achat • Liberté totale : Vous êtes propriétaire du véhicule et pouvez l'utiliser sans restriction. • Coûts à long terme réduits : Une fois le prêt remboursé (si achat à crédit), le coût mensuel diminue. • Valeur de revente : Possibilité de revendre le véhicule pour financer un futur achat. 1.2 Inconvénients de l'achat • Investissement initial élevé : Coût d'achat et d'immatriculation parfois importants. • Dépréciation rapide : Une voiture perd en moyenne 20 % de sa valeur la première année. • Coûts d'entretien et de réparation : À la charge du propriétaire. 1.3 Neuf ou occasion ? • Voiture

In [4]:
import re
import mistune
from bs4 import BeautifulSoup

def extract_text_from_md(md_path):
    with open(md_path, "r", encoding="utf-8") as f:
        md_content = f.read()
    html = mistune.html(md_content)
    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text(separator=" ")
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

md = extract_text_from_md('./ref/doc.md')
print(md)

Create the .env.local File with Secrets M-Motors: A Success Story M-Motors was founded in 1987 as a specialist in selling used vehicles. The company has achieved significant success and reputation. After 30 years, it became one of the top 10 companies nationwide. This success stems from several factors, all centered around customer satisfaction. A Diverse Offering and High-Quality Customer Service The company offers a wide range of: Brands Models Engine types Mileage options Prices This allows it to cater to all customer needs and budgets. Additionally, the vehicles are of high quality, ensuring good mechanical condition and regular maintenance to guarantee reliability and safety. This includes: Thorough technical inspections Repairs and refurbishments Warranty options The company also has a commercial service and after-sales service that support customers with personalized advice. The goal is to provide high-quality customer service by understanding their needs and guiding them toward

In [ ]:
import pandas as pd

def extract_text_from_csv(csv_path):
    df = pd.read_csv(csv_path, encoding='utf-8')
    text_data = df.astype(str).apply(lambda x: " ".join(x), axis=1)
    return " ".join(text_data)

csv = extract_text_from_csv('./ref/doc.csv')
print(csv)

"Introduction nan L'achat ou la location d'une voiture dépend du budget, de l'usage et des coûts d'entretien. Achat Avantages Liberté totale, coûts réduits à long terme, possibilité de revente. Achat Inconvénients Investissement initial élevé, dépréciation rapide, coûts d'entretien. Achat Neuf ou Occasion Neuf : plus cher mais garantie. Occasion : moins cher mais vérifier l'historique. Achat Critères Budget, type de carburant, usage, espace et confort. Location Avantages Coût mensuel prévisible, accès à des véhicules récents, pas de souci de revente. Location Inconvénients Absence de propriété, restrictions d'utilisation, dépôt de garantie. Location Types LOA : achat possible en fin de contrat. LLD : pas d'achat. Conclusion nan L'achat est préférable pour le long terme, la location pour changer régulièrement de voiture."

In [ ]:
import orjson

def extract_text_from_json(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = orjson.loads(f.read())

    def recursive_extract(data, prefix=""):
        if isinstance(data, dict):
            return " ".join([recursive_extract(v, f"{prefix}{k}: ") for k, v in data.items()])
        elif isinstance(data, list):
            return " ".join([recursive_extract(v, prefix) for v in data])
        elif isinstance(data, (int, float, str)):
            return f"{prefix}{data}"
        return ""

    return recursive_extract(data).strip()

json = extract_text_from_json('./ref/doc.json')
print(json)

"Introduction: L'achat ou la location d'une voiture dépend du budget, de l'usage et des coûts d'entretien. Achat: Avantages: Liberté totale Achat: Avantages: Coûts réduits à long terme Achat: Avantages: Possibilité de revente Achat: Inconvénients: Investissement initial élevé Achat: Inconvénients: Dépréciation rapide Achat: Inconvénients: Coûts d'entretien Achat: Neuf ou Occasion: Neuf: Plus cher mais garantie constructeur et technologies modernes. Achat: Neuf ou Occasion: Occasion: Moins cher mais attention à l'état général et à l'historique du véhicule. Achat: Critères: Budget Achat: Critères: Type de carburant Achat: Critères: Usage Achat: Critères: Espace et confort Location: Avantages: Coût mensuel prévisible Location: Avantages: Accès à des véhicules récents Location: Avantages: Pas de souci de revente Location: Inconvénients: Absence de propriété Location: Inconvénients: Restrictions d'utilisation Location: Inconvénients: Dépôt de garantie et frais supplémentaires Location: Type

In [ ]:
import lxml.etree as ET

def extract_text_from_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    text = " ".join([elem.text for elem in root.iter() if elem.text])
    return text.strip()

xml = extract_text_from_xml('./ref/doc.xml')
print(xml)

"L'achat ou la location d'une voiture dépend du budget, de l'usage et des coûts d'entretien. Liberté totale Coûts réduits à long terme Possibilité de revente Investissement initial élevé Dépréciation rapide Coûts d'entretien Plus cher mais garantie constructeur et technologies modernes. Moins cher mais attention à l'état général et à l'historique du véhicule. Budget Type de carburant Usage Espace et confort Coût mensuel prévisible Accès à des véhicules récents Pas de souci de revente Absence de propriété Restrictions d'utilisation Dépôt de garantie et frais supplémentaires Achat possible en fin de contrat. Pas d'achat possible, seulement la location longue durée. L'achat est préférable pour le long terme, la location pour changer régulièrement de voiture."

## Netoyage Avancé

In [ ]:
def remove_invisible_ascii(text):
    return ''.join(c for c in text if c.isprintable())

In [74]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,4.00,4.00,4.00,4.00,4.00
mean,2.50,3526.25,703.75,9.75,881.56
std,1.29,1104.34,90.74,1.89,276.09
min,1.00,1949.00,585.00,7.00,487.25
25%,1.75,3228.50,678.75,9.25,807.12
50%,2.50,3838.00,712.00,10.50,959.50
75%,3.25,4135.75,737.00,11.00,1033.94
max,4.00,4480.00,806.00,11.00,1120.00


In [79]:
!python -m spacy download en_core_web_sm
!python -m spacy download fr_core_news_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     -------------- ------------------------- 4.7/12.8 MB 28.6 MB/s eta 0:00:01
     ----------------------------------- --- 11.8/12.8 MB 32.0 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 28.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
     ---------------------------------------- 0.0/16.3 MB ? eta -:--:--
     ----------- ---------------------------- 4.7/16.3 MB 28.6 MB/s eta 0:00:01
     ------------------------------ -------- 12.8/16.3 MB 32.3 MB/s eta 0:00:01
     --------------------------------------- 16.3/16.3 MB 30.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [87]:



from spacy.lang.fr import French # see https://spacy.io/usage for install instructions

nlp = French()

nlp.add_pipe("sentencizer")

doc = nlp("This is a sentence. This another sentence.")
assert len(list(doc.sents)) == 2

list(doc.sents)

[This is a sentence., This another sentence.]

In [ ]:
from ollama import chat
from ollama import ChatResponse

query_text = 'Why is the sky blue?'

print(f"Testing with model llama3.2:1b")

response: ChatResponse = chat(
    model='llama3.2:1b',
    messages=[{'role': 'user', 'content': query_text}],
    stream=False
)
print(response['message']['content'])

- `seed` : Nombre aléatoire pour l'initialisation du générateur de nombres aléatoires  
  min : 0 / max : 2147483647 (selon les limites des entiers 32 bits) / default : Aucune valeur définie (généré aléatoirement)

- `num_predict` : Nombre maximal de tokens à prédire  
  min : 1 / max : Limite du vocabulaire du modèle (environ 50000 pour Llama3.2:1b) / default : 256

- `top_k` : Nombre de tokens parmi les meilleurs à choisir lors de la génération  
  min : 1 / max : Taille du vocabulaire du modèle (environ 50000 pour Llama3.2:1b) / default : 50

- `top_p` : Contrôle la diversité des prédictions en fonction de la probabilité cumulative des tokens  
  min : 0.0 / max : 1.0 / default : 1.0

- `min_p` : Utilisé pour ajuster la probabilité minimale des tokens en fonction de la probabilité du token de top  
  min : 0.0 / max : 1.0 / default : 0.0

- `num_ctx` : Nombre de contextes (tokens) à prendre en compte pour l'inférence  
  min : 1 / max : 4096 / default : 4096

- `temperature` : Niveau de créativité de la réponse générée. Une température plus basse produit des réponses plus déterministes  
  min : 0.0 / max : 2.0 / default : 1.0

- `repeat_penalty` : Force avec laquelle les répétitions sont pénalisées pendant la génération  
  min : 1.0 / max : 2.0 / default : 1.1

- `repeat_last_n` : Nombre de tokens passés à prendre en compte pour la pénalisation des répétitions  
  min : 1 / max : 100 / default : 64

- `num_gpu` : Nombre de GPU à utiliser pour l'inférence  
  min : 1 / max : Nombre total de GPU disponibles / default : 1

- `stop` : Liste de chaînes de caractères où le modèle s'arrête dès l'une d'elles est rencontrée  
  min : Aucun / max : Liste de chaînes / default : Aucun

In [ ]:
from ollama import chat
from ollama import ChatResponse

query_text = 'Qui est le dernier président des Etat-Unis?'

params_list = [
    {
        'temperature': 1.5,
        'top_k': 100,
        'top_p': 0.9,
        'num_predict': 512,
        'num_ctx': 2048,
        'repeat_penalty': 1.1,
        'repeat_last_n': 64,
        'num_gpu': 1,
        'stop': ['\n'],
        'seed': 12345
    },
    {
        'temperature': 0.3,
        'top_k': 20,
        'top_p': 0.8,
        'num_predict': 150,
        'num_ctx': 1024,
        'repeat_penalty': 1.2,
        'repeat_last_n': 32,
        'num_gpu': 1,
        'stop': ['.'],
        'seed': 67890
    },
    {
        'temperature': 0.8,
        'top_k': 50,
        'top_p': 0.95,
        'num_predict': 256,
        'num_ctx': 1024,
        'repeat_penalty': 1.0,
        'repeat_last_n': 32,
        'num_gpu': 1,
        'stop': ['\n'],
        'seed': 112233
    }
]
for params in params_list:
    print(f"Testing with: temperature={params['temperature']} | top_p={params['top_p']}")

    response: ChatResponse = chat (
        model='llama3.2:1b',
        messages=[{'role': 'user', 'content': query_text}],
        stream=False,
        options=params 
    )
    print(response['message']['content'], '\n\n')

#### Configuration 1 

```python
  'temperature': 1.5,
  'top_k': 100,
  'top_p': 0.9,
  'num_predict': 512,
  'num_ctx': 2048,
  'repeat_penalty': 1.1,
  'repeat_last_n': 64,
  'num_gpu': 1,
  'stop': ['\n'],
  'seed': 12345
```

##### Résultat : 

Actuellement, Barack Obama, qui était le 44e président des États-Unis, c'est-à-dire depuis janvier 2017, succède à Joe Biden, qui est actuellement le 46e président. Barack Obama est un politologue et scénariste américain célèbre pour son campagnede présidentielle en 2008 et sa nomination comme 44e président des États-Unis par le conseil de dirigeance républicain en novembre 2007. 

##### Conclusion : 

**Créative et longue**

---

#### Configuration 2

```python
  'temperature': 0.3,
  'top_k': 20,
  'top_p': 0.8,
  'num_predict': 150,
  'num_ctx': 1024,
  'repeat_penalty': 1.2,
  'repeat_last_n': 32,
  'num_gpu': 1,
  'stop': ['.'],
  'seed': 67890
```
##### Résultat : 

Testing with: temperature=0.3 | top_p=0.8
Le dernier président des États-Unis est Joe Biden 

##### Conclusion : 

**Déterministe et concis**

---

#### Configuration 3

```python
  'temperature': 1.5,
  'top_k': 100,
  'top_p': 0.9,
  'num_predict': 512,
  'num_ctx': 2048,
  'repeat_penalty': 1.1,
  'repeat_last_n': 64,
  'num_gpu': 1,
  'stop': ['\n'],
  'seed': 12345
```

##### Résultat : 

Testing with: temperature=0.8 | top_p=0.95
En fait, les États-Unis ont eu plusieurs présidents depuis 1789 jusqu'à l'actualité. L'État actuel du président des États-Unis est Joe Biden, qui a occupé ce titre depuis le 20 décembre 2021. 

##### Conclusion : 

**Equilibré**

---